In [8]:
from google.colab import files
uploaded = files.upload()

Saving brain_tumor_dataset.zip to brain_tumor_dataset.zip


In [9]:
import zipfile
with zipfile.ZipFile("/content/brain_tumor_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("destination_folder")

In [10]:
from PIL import Image
import os
import hashlib
import cv2 #module from the OpenCV (Open Source Computer Vision) library, . It provides functions for reading, writing, and manipulating images .  cv2 module is used to read the image files from the dataset directory, resize them to a fixed size, convert them to RGB color space, and normalize their pixel values to range [0, 1].
import numpy as np
from skimage import io, exposure
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import BatchNormalization
from sklearn.metrics import classification_report
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [11]:
categories = os.listdir(r"/content/destination_folder/brain_tumor_dataset")
num_categories = len(categories)
print("Number of categories:", num_categories)

Number of categories: 2


In [12]:
category_to_label = {}
for i, category in enumerate(categories):
    category_to_label[category] = i
print("Category to label mapping:", category_to_label)

Category to label mapping: {'no': 0, 'yes': 1}


In [13]:
# Define a function to remove corrupt images
def remove_corrupt_images(directory):
    # Loop through all files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        try:
            # Attempt to open the image file
            with Image.open(filepath) as img:
                img.verify()
        except (IOError, SyntaxError) as e:
            # Delete the corrupt image file
            os.remove(filepath)
            print("Removed corrupt image:", filepath)

remove_corrupt_images(r'/content/destination_folder/brain_tumor_dataset/yes')

remove_corrupt_images(r'/content/destination_folder/brain_tumor_dataset/no')

In [14]:
target_size = (224, 224)
# Define a function to resize images in a directory
def resize_images(directory):

    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        # Check if the file is an image
        if filepath.endswith(".jpg") or filepath.endswith(".png") or filepath.endswith(".jpeg"):

            with Image.open(filepath) as img:
                # Resize the image to the target size
                resized_img = img.resize(target_size)

                # Convert RGBA to RGB if the image has an alpha channel
                if resized_img.mode == 'RGBA':
                    resized_img = resized_img.convert('RGB')


                resized_img.save(filepath)



resize_images(r'/content/destination_folder/brain_tumor_dataset/yes')

resize_images(r'/content/destination_folder/brain_tumor_dataset/no')

In [15]:
# Define a function to normalize images in a directory
def normalize_images(directory):

    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        if filepath.endswith(".jpg") or filepath.endswith(".png"):

            img = io.imread(filepath)

            # Normalize the image
            normalized_img = exposure.rescale_intensity(img, in_range='image', out_range=(0, 1))

normalize_images(r'/content/destination_folder/brain_tumor_dataset/yes')

normalize_images(r'/content/destination_folder/brain_tumor_dataset/no')

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=45,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = 0.3)
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.3)

In [17]:
training_set = train_datagen.flow_from_directory(r"/content/destination_folder/brain_tumor_dataset",
                                                 target_size=target_size,
                                                batch_size=32,
                                                 class_mode='categorical',
                                                subset='training')

validation_set = test_datagen.flow_from_directory(r"/content/destination_folder/brain_tumor_dataset",
                                                  target_size=target_size,
                                                 batch_size=32,
                                      class_mode='categorical',
                                               shuffle = False,
                                           subset='validation')

Found 177 images belonging to 2 classes.
Found 75 images belonging to 2 classes.


In [18]:
# ANN ALGORITHM
model = Sequential()
model.add(Flatten(input_shape=(224, 224, 3)))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(training_set,
          epochs=100,
          validation_data=validation_set)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 283ms/step - accuracy: 0.5339 - loss: 0.8075 - val_accuracy: 0.6133 - val_loss: 4.6377
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 264ms/step - accuracy: 0.5769 - loss: 0.8121 - val_accuracy: 0.6133 - val_loss: 3.4575
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 222ms/step - accuracy: 0.6676 - loss: 0.6758 - val_accuracy: 0.6133 - val_loss: 2.9290
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 217ms/step - accuracy: 0.6228 - loss: 0.6293 - val_accuracy: 0.6133 - val_loss: 2.2954
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 218ms/step - accuracy: 0.6248 - loss: 0.6317 - val_accuracy: 0.6133 - val_loss: 1.8441
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 218ms/step - accuracy: 0.5952 - loss: 0.6657 - val_accuracy: 0.6133 - val_loss: 1.5773
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 258ms/step - accuracy: 0.6635 - loss: 0.6087 - val_accuracy: 0.6133 - val_loss: 1.4081
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 217ms/step - accuracy: 0.7249 - loss: 0.5617 - val_accuracy: 0.6267 - val_loss: 1.2

In [19]:
test_generator = test_datagen.flow_from_directory(
    directory=r"/content/destination_folder/brain_tumor_dataset",
    target_size=target_size,
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy for ANN ALGORITHM :", test_acc)

Found 252 images belonging to 2 classes.
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.7303 - loss: 0.5287
Test accuracy for ANN ALGORITHM : 0.829365074634552


In [20]:
# Get the predicted labels for the validation set
y_pred = model.predict(validation_set)
y_pred = np.argmax(y_pred, axis=1)

# Get the true labels for the validation set
y_true = validation_set.classes

# Compute the precision, recall, f1-score, and support
target_names = ['no', 'yes']

print("Report for ANN ALGORITHM ")

print(classification_report(y_true, y_pred, target_names=target_names))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Report for ANN ALGORITHM 
              precision    recall  f1-score   support

          no       0.79      0.52      0.62        29
         yes       0.75      0.91      0.82        46

    accuracy                           0.76        75
   macro avg       0.77      0.72      0.72        75
weighted avg       0.77      0.76      0.75        75



In [21]:
import requests
# Load the image
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Hirnmetastase_MRT-T1_KM.jpg/220px-Hirnmetastase_MRT-T1_KM.jpg"
response = requests.get(url)
img_array = np.frombuffer(response.content, np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

# Preprocess the image
target_size = (224, 224)  # Specify the target size
img = cv2.resize(img, target_size)
img = img.astype('float32') / 255.0
img = np.expand_dims(img, axis=0)

# Make the prediction
prediction = model.predict(img)

# Print the prediction
if prediction[0][0] > 0.5:
    print('The model predicts that the image shows no tumor.')
else:
    print('The model predicts that the image shows yes tumor.')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
The model predicts that the image shows yes tumor.


In [22]:
# Load the image
img = cv2.imread(r"/content/destination_folder/brain_tumor_dataset/no/10 no.jpg")

# Preprocess the image

img = cv2.resize(img, target_size)
img = img.astype('float32') / 255.0
img = np.expand_dims(img, axis=0)

#Make the prediction
prediction = model.predict(img)

# Print the prediction
if prediction[0][0] > 0.5:
       print('The model predicts that the image shows no tumor.')
else:
       print('The model predicts that the image shows yes tumor.')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
The model predicts that the image shows no tumor.


In [23]:
# CNN ALGORITHM
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224,224, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(training_set,
          epochs=100,
          validation_data=validation_set)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.5538 - loss: 0.9699 - val_accuracy: 0.7467 - val_loss: 0.5670
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.7493 - loss: 0.5905 - val_accuracy: 0.7333 - val_loss: 0.5784
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 29s 4s/step - accuracy: 0.6961 - loss: 0.5762 - val_accuracy: 0.7333 - val_loss: 0.5610
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.7471 - loss: 0.5630 - val_accuracy: 0.7600 - val_loss: 0.5786
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 29s 4s/step - accuracy: 0.7611 - loss: 0.5610 - val_accuracy: 0.7600 - val_loss: 0.5624
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 45s 5s/step - accuracy: 0.8244 - loss: 0.4555 - val_accuracy: 0.7200 - val_loss: 0.5782
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 33s 6s/step - accuracy: 0.7665 - loss: 0.4879 - val_accuracy: 0.7467 - val_loss: 0.5745
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 38s 5s/step - accuracy: 0.7578 - loss: 0.5217 - val_accuracy: 0.7467 - val_loss: 0.5821


In [25]:
test_generator = test_datagen.flow_from_directory(
    directory=r"/content/destination_folder/brain_tumor_dataset",
    target_size=target_size,
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy for CNN ALGORITHM :", test_acc)

Found 252 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.8371 - loss: 0.4412
Test accuracy for CNN ALGORITHM : 0.9047619104385376


In [26]:
# Get the predicted labels for the validation set
y_pred = model.predict(validation_set)
y_pred = np.argmax(y_pred, axis=1)

# Get the true labels for the validation set
y_true = validation_set.classes

# Compute the precision, recall, f1-score, and support
target_names = ['no', 'yes']

print("Report for CNN ALGORITHM ")

print(classification_report(y_true, y_pred, target_names=target_names))

2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 958ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 684ms/step
Report for CNN ALGORITHM 
              precision    recall  f1-score   support

          no       0.74      0.59      0.65        29
         yes       0.77      0.87      0.82        46

    accuracy                           0.76        75
   macro avg       0.75      0.73      0.74        75
weighted avg       0.76      0.76      0.75        75



In [31]:
import requests
# Load the image
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Hirnmetastase_MRT-T1_KM.jpg/220px-Hirnmetastase_MRT-T1_KM.jpg"
response = requests.get(url)
img_array = np.frombuffer(response.content, np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

# Preprocess the image
target_size = (224, 224)  # Specify the target size
img = cv2.resize(img, target_size)
img = img.astype('float32') / 255.0
img = np.expand_dims(img, axis=0)

# Make the prediction
prediction = model.predict(img)

# Print the prediction
if prediction[0][0] > 0.5:
    print('The model predicts that the image shows no tumor.')
else:
    print('The model predicts that the image shows yes tumor.')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
The model predicts that the image shows yes tumor.


In [32]:
model = Sequential()
model.add(Flatten(input_shape=(224, 224, 3)))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))

# Compile the model with a custom loss function for SVM
model.compile(optimizer=SGD(), loss='hinge', metrics=['accuracy'])

# Train the SVM model using fit_generator
history = model.fit(training_set,
          epochs=100,
          validation_data=validation_set)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 171ms/step - accuracy: 0.5307 - loss: 0.9837 - val_accuracy: 0.4267 - val_loss: 1.0509
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - accuracy: 0.5994 - loss: 0.9199 - val_accuracy: 0.6133 - val_loss: 0.8612
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.6164 - loss: 0.9195 - val_accuracy: 0.6133 - val_loss: 0.8503
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.6367 - loss: 0.8695 - val_accuracy: 0.6533 - val_loss: 0.8429
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.6688 - loss: 0.8659 - val_accuracy: 0.6667 - val_loss: 0.8288
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.6962 - loss: 0.8469 - val_accuracy: 0.7467 - val_loss: 0.8273
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - accuracy: 0.6779 - loss: 0.8828 - val_accuracy: 0.6667 - val_loss: 0.8318
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.7467 - loss: 0.8375 - val_accuracy: 0.6667 - val_

In [34]:
test_generator = test_datagen.flow_from_directory(
    directory=r"/content/destination_folder/brain_tumor_dataset",
    target_size=target_size,
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy for SVM ALGORITHM :", test_acc)

Found 252 images belonging to 2 classes.
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.6910 - loss: 0.8191
Test accuracy for SVM ALGORITHM : 0.7936508059501648


In [35]:
# Get the predicted labels for the validation set
y_pred = model.predict(validation_set)
y_pred = np.argmax(y_pred, axis=1)

# Get the true labels for the validation set
y_true = validation_set.classes

# Compute the precision, recall, f1-score, and support
target_names = ['no', 'yes']

print("Report for SVM ALGORITHM ")

print(classification_report(y_true, y_pred, target_names=target_names))

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step
Report for SVM ALGORITHM 
              precision    recall  f1-score   support

          no       0.81      0.59      0.68        29
         yes       0.78      0.91      0.84        46

    accuracy                           0.79        75
   macro avg       0.79      0.75      0.76        75
weighted avg       0.79      0.79      0.78        75



In [36]:
import requests
# Load the image
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Hirnmetastase_MRT-T1_KM.jpg/220px-Hirnmetastase_MRT-T1_KM.jpg"
response = requests.get(url)
img_array = np.frombuffer(response.content, np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

# Preprocess the image
target_size = (224, 224)  # Specify the target size
img = cv2.resize(img, target_size)
img = img.astype('float32') / 255.0
img = np.expand_dims(img, axis=0)

# Make the prediction
prediction = model.predict(img)

# Print the prediction
if prediction[0][0] > 0.5:
    print('The model predicts that the image shows no tumor.')
else:
    print('The model predicts that the image shows yes tumor.')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
The model predicts that the image shows yes tumor.
